# plot RSM #

In [ ]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Text, Layout, FileUpload
import matplotlib.pyplot as plt
import numpy as np
import rsmPlot
import scipy.interpolate as si


In [ ]:
uploader = widgets.FileUpload(
    accept='raw',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description = 'Upload RAW file'
)
display(uploader)

In [ ]:
button_rsm = widgets.Button(description='Plot RSM ', layout=Layout(height='auto', width='full'))

cmap_rsm = widgets.Dropdown(
    options=['turbo', 'plasma', 'viridis', 'jet', 'gray'],
    value='turbo',
    description='Color Map',
    disabled=False,
)

Vmax_rsm = widgets.Text(value='None', placeholder='Type something', description='Vmax:',  disabled=False)
Log_rsm = widgets.Checkbox(value=True, description='Log intensity', disabled=False)

items_auto = [button_rsm, cmap_rsm, Log_rsm, Vmax_rsm ]


box_layout = Layout(display='flex', flex_flow='row',  width='70%')
box_auto = widgets.Box(children=items_auto, layout=box_layout)

plt.ioff()
fig = plt.figure()
fig.canvas.header_visible = False
fig.canvas.layout.min_height = '500px'



def plot_rsm(b):
    fig.clear(True) 
    plt.figure(fig)
    try:
        vmaxI= float(Vmax_rsm.value)
    except:
        vmaxI=None

    Cvo = rsmPlot.XRDfile(uploader.value[0].content)
    X, Y, intensity, cbarlabel, theta, Dtheta, counts = Cvo.plot_rsm(log=Log_rsm.value, vmax=vmaxI, output=True)

    zz = plt.contourf(X, Y, intensity, levels=500)
    cbar = plt.colorbar()
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")
    plt.xlabel(r'$\theta$($\degree$)')
    plt.ylabel(r'$2\theta$($\degree$)')
    plt.title(f"{Cvo.info['LAYER']}/{Cvo.info['SUBSTRATE']}")
    fig.canvas.draw()
    plt.tight_layout() 
    func = si.RectBivariateSpline(theta, Dtheta, counts.T)
    def fmt(x, y):
        z = np.take(func(x, y), 0)
        return 'x={x:.5f}  y={y:.5f}  counts={z:.5f}'.format(x=x, y=y, z=z)
    fig.axes[0].format_coord = fmt


    
button_rsm.on_click(plot_rsm)

widgets.VBox([box_auto, fig.canvas])

In [ ]:
button_rsmQ = widgets.Button(description='Plot RSM Qvector', layout=Layout(height='auto', width='full'))

cmap_rsmQ = widgets.Dropdown(
    options=['turbo', 'plasma', 'viridis', 'jet', 'gray'],
    value='turbo',
    description='Color Map',
    disabled=False, layout=Layout(height='auto', width='auto', padx=10)
)



Vmax_rsmQ = widgets.Text(value='None', placeholder='Type something', description='Vmax\n(count):',  
                         disabled=False, layout=Layout(height='auto', width='150px', padx=10))
Log_rsmQ = widgets.Checkbox(value=True, description='Log intensity', disabled=False)

items_autoQ = [button_rsmQ, cmap_rsmQ, Vmax_rsmQ, Log_rsmQ ]


box_layoutQ = Layout(display='flex', flex_flow='row', align_items='stretch', width='80%')
box_autoQ = widgets.Box(children=items_autoQ, layout=box_layoutQ)

plt.ioff()
figQ = plt.figure()
fig.canvas.header_visible = False
figQ.canvas.layout.min_height = '500px'


def plot_rsmQ(b):
    figQ.clear(True) 
    plt.figure(figQ)
    try:
        vmaxI= float(Vmax_rsmQ.value)
    except:
        vmaxI=None
    
    CvoQ = rsmPlot.XRDfile(uploader.value[0].content)
    qpara, qperp, intensity, cbarlabel = CvoQ.plot_rsmQ(log=Log_rsmQ.value, vmax=vmaxI, levels=500, output=True)
    
    plt.contourf(qpara, qperp, intensity, cmap=cmap_rsmQ.value, levels=500)
    cbarQ = plt.colorbar()
    cbarQ.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")
    plt.xlabel(r'$q\parallel$($\AA^{-1}$)')
    plt.ylabel(r'$q\perp$($\AA^{-1}$)')
    plt.title(f"{CvoQ.info['LAYER']}/{CvoQ.info['SUBSTRATE']}")
    plt.tight_layout() 
    figQ.canvas.draw()


button_rsmQ.on_click(plot_rsmQ)


widgets.VBox([box_autoQ, figQ.canvas])